![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Comparación de modelos de clasificación

## Aprendizaje Automático Aplicado

## Maestría en Ciencia de Datos

**Ivo Jiménez** y **Julio Waissman**, 2022

[Abrir en google Colab](https://colab.research.google.com/github/mcd-unison/aaa-curso/blob/main/ejemplos/plot_classifier_comparison.ipynb)


Libreta tomada de [la documentación de `sklearn`](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html). De acuerdo a los autores originales (Gaël Varoquaux, Andreas Müller, modificado por Jaques Grobler):


> A comparison of a several classifiers in scikit-learn on synthetic datasets.
> The point of this example is to illustrate the nature of decision boundaries
> of different classifiers.
> This should be taken with a grain of salt, as the intuition conveyed by
> these examples does not necessarily carry over to real datasets.
> 
> Particularly in high-dimensional spaces, data can more easily be separated
> linearly and the simplicity of classifiers such as naive Bayes and linear SVMs
> might lead to better generalization than is achieved by other classifiers.
> 
> The plots show training points in solid colors and testing points
> semi-transparent. The lower right shows the classification accuracy on the test
> set.

En nuestro caso, lo que nos parece más importante de resaltar es la API estandarizada que se utiliza en `sklearn`
para diferentes modelos de aprendizaje. Vamos a dejar la misma libreta pero por partes para ir discutiendo que es lo que se hace. 

Vamos a iniciar cargando las librerías necesarias:

In [ ]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

%matplotlib inline

# Modulos generales
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Para dividir los conjuntos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split

# Para escalar los datos
from sklearn.preprocessing import StandardScaler

# Para crear conjuntos de datos sintéticos
from sklearn.datasets import make_moons, make_circles, make_classification


Y ahora vamos a cargar unos 9 modelos de ML para clasificación. Por el momento no importa si los conocemos o no, solo para ver como se pueden usar:

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

¿Que son? ¿Que hacen?, vamos a generar una lista de objetos que ya es un modelo particular, y otra lista con sus nombres
(para efectos de presentación)

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

# Y ahora los nombres
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

Listo, ya tenemos 10 modelos de aprendizaje diferentes. Ahora nos faltan los datos en los que vamos a aplicar los modelos.

Para los datos vamos a crear 3 conjuntos sintéticos en solo dos dimensiones para que podamos visualizar:

1. Un conjunto usando un generador de lunas con dos clases (como el ying y el yang), a las que agregaremos ruido.
2. Un conjunto *envuelto* en otro conjunto (circulos concentricos) pero con ruido
3. Un conjunto linealmente separable, pero con ruido en los datos

Vamos a generar entonces los conjuntos de datos:

In [ ]:
# Datos sintéticos en forma de lunas
Xm, ym = make_moons(noise=0.3, random_state=0)

# Datos sintéticos en forma de circulos concentricos
Xc, yc = make_circles(noise=0.2, factor=0.5, random_state=1)

# Datos sintéticos de un linealmente separable, mas ruido
Xl, yl = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
Xl += 2 * rng.uniform(size=Xl.shape)

datasets = [
    (Xm, ym),
    (Xc, yc),
    (Xl, yl)
]

Y aqui va a suceder la mágia. Bueno esta parte del código no se puede separar por partes, así que la vamos a ir revisando juntos paso a paso. Lo más importante ees ver como hay una serie de métodos genericos que nos permiten usar mñodelos de aprendizaje sin siquiera saber de que se tratan o que es lo que hacen. 

Eso lo hace muy atractivo y al mismo tiempo muy peligroso, vamos a verlo.

In [ ]:
h = 0.02  # step size in the mesh

figure = plt.figure(figsize=(27, 9))
i = 1
# itera sobre los 3 conjuntos de datos
for ds_cnt, ds in enumerate(datasets):
    
    # Estandariza los datos de entrada y separa en conjunto de entrenamiento y prueba
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

    # Calcula los valores minimo y máximo a clasificar
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    # Genera datos para clasificar en toda la gráfica con incrementos de h
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h), 
        np.arange(y_min, y_max, h)
    )

    # Grafica el conjunto de datos
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(
        X_train[:, 0], 
        X_train[:, 1], 
        c=y_train, 
        cmap=cm_bright, 
        edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0], 
        X_test[:, 1], 
        c=y_test, 
        cmap=cm_bright, 
        alpha=0.6, edgecolors="k"
    )
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    
    # Y ahora listos para la siguiente gráfica
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):

        # Clasifica con datos de entrenamiento y calcula el score con los de prueba
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)

        # Put the result into a color plot
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        ax.contourf(
            xx, 
            yy, 
            Z, 
            cmap=cm, 
            alpha=0.8
        )

        # Plot the training points
        ax.scatter(
            X_train[:, 0], 
            X_train[:, 1], 
            c=y_train, 
            cmap=cm_bright, 
            edgecolors="k"
        )

        # Plot the testing points
        ax.scatter(
            X_test[:, 0],
            X_test[:, 1],
            c=y_test,
            cmap=cm_bright,
            edgecolors="k",
            alpha=0.6,
        )

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(
            xx.max() - 0.3,
            yy.min() + 0.3,
            ("%.2f" % score).lstrip("0"),
            size=15,
            horizontalalignment="right",
        )
        i += 1

plt.tight_layout()
plt.show()